In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Adrenocortical_Cancer"
cohort = "GSE68950"

# Input paths
in_trait_dir = "../../input/GEO/Adrenocortical_Cancer"
in_cohort_dir = "../../input/GEO/Adrenocortical_Cancer/GSE68950"

# Output paths
out_data_file = "../../output/preprocess/Adrenocortical_Cancer/GSE68950.csv"
out_gene_data_file = "../../output/preprocess/Adrenocortical_Cancer/gene_data/GSE68950.csv"
out_clinical_data_file = "../../output/preprocess/Adrenocortical_Cancer/clinical_data/GSE68950.csv"
json_path = "../../output/preprocess/Adrenocortical_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
```python
import pandas as pd
import os
import json
import numpy as np
from typing import Optional, Callable, Dict, Any

# Load the clinical data from the raw data file
try:
    # Try loading from the matrix file directly instead of pkl
    matrix_file = os.path.join(in_cohort_dir, "matrix.csv")
    if os.path.exists(matrix_file):
        clinical_data = pd.read_csv(matrix_file, index_col=0)
        print(f"Matrix file loaded successfully from {matrix_file}")
    else:
        # Try to find any available data files
        data_files = [f for f in os.listdir(in_cohort_dir) if f.endswith(('.txt', '.csv'))]
        if data_files:
            matrix_file = os.path.join(in_cohort_dir, data_files[0])
            clinical_data = pd.read_csv(matrix_file, sep='\t' if matrix_file.endswith('.txt') else ',', index_col=0)
            print(f"Data file loaded successfully from {matrix_file}")
        else:
            raise FileNotFoundError(f"No data files found in {in_cohort_dir}")
    
    # Extract and examine sample characteristics
    if '!Sample_characteristics_ch1' in clinical_data.index:
        # Characteristics are in the rows
        sample_chars = clinical_data.loc[['!Sample_characteristics_ch1']]
        
        # Transpose if needed to have samples as rows and characteristics as columns
        if len(sample_chars.columns) > len(sample_chars.index):
            # Already in the right format
            characteristics_df = sample_chars
        else:
            characteristics_df = sample_chars.T
        
        print("Sample characteristics found in the index")
    elif '!Sample_characteristics_ch1' in clinical_data.columns:
        # Characteristics are in the columns
        characteristics_df = clinical_data[['!Sample_characteristics_ch1']]
        print("Sample characteristics found in the columns")
    else:
        # Look for other potential characteristic headers
        potential_headers = [col for col in clinical_data.columns if 'characteristics' in col.lower()]
        if potential_headers:
            characteristics_df = clinical_data[potential_headers]
            print(f"Using alternative characteristic headers: {potential_headers}")
        else:
            # Check if this is a standard GEO format with characteristics in multiple rows
            char_rows = [i for i, idx in enumerate(clinical_data.index) if 'characteristics' in str(idx).lower()]
            if char_rows:
                characteristics_df = clinical_data.iloc[char_rows]
                print(f"Found characteristics in rows: {char_rows}")
            else:
                raise ValueError("Sample characteristics not found in the data")
    
    print("\nData structure:")
    print(f"Shape: {clinical_data.shape}")
    print(f"Index: {list(clinical_data.index)[:5]}...")
    print(f"Columns: {list(clinical_data.columns)[:5]}...")
    
    # Print a sample of the characteristics
    print("\nSample characteristics preview:")
    print(characteristics_df.head())
    
    # Identify trait, age, and gender information from the characteristics
    trait_row = None
    age_row = None
    gender_row = None
    
    # Collect unique values for each row to analyze content
    unique_values = {}
    
    # Depending on the structure, extract sample characteristics
    if isinstance(characteristics_df, pd.DataFrame):
        # If we have multiple characteristic rows/cols
        for i, row in enumerate(characteristics_df.index):
            if isinstance(characteristics_df, pd.DataFrame) and len(characteristics_df.columns) > 0:
                values = []
                for col in characteristics_df.columns:
                    val = characteristics_df.loc[row, col]
                    if pd.notna(val):
                        values.append(str(val))
                if values:
                    unique_values[i] = "; ".join(set(values))
    
    # If no values were extracted using the method above, try an alternative approach
    if not unique_values:
        # Try to extract directly from the matrix file
        for i in range(min(20, len(clinical_data))):  # Check first 20 rows for characteristics
            if i < len(clinical_data.index):
                row_name = clinical_data.index[i]
                if isinstance(row_name, str) and "characteristics" in row_name.lower():
                    values = clinical_data.iloc[i].astype(str).tolist()
                    unique_values[i] = "; ".join(set(values))
    
    print("\nUnique values for potential characteristic rows:")
    for key, value in unique_values.items():
        print(f"Row {key}: {value}")
    
    # Analyze the unique values to identify trait, age, and gender
    for row_idx, values in unique_values.items():
        values_lower = values.lower()
        
        # Identify trait information
        if any(term in values_lower for term in ["diagnosis", "tissue", "tumor", "carcinoma", "status", "histology", "sample type", "sample_type", "disease"]):
            trait_row = row_idx
            print(f"Found trait row: {row_idx} - {values}")
        
        # Identify age information
        if "age" in values_lower:
            age_row = row_idx
            print(f"Found age row: {row_idx} - {values}")
        
        # Identify gender/sex information
        if any(term in values_lower for term in ["gender", "sex"]):
            gender_row = row_idx
            print(f"Found gender row: {row_idx} - {values}")
        
except Exception as e:
    print(f"Error processing data: {e}")
    # Set default values as we couldn't analyze the data
    unique_values = {}
    trait_row = None
    age_row = None
    gender_row = None

# 1. Gene Expression Data Availability
# For GEO datasets with GSE prefix, we generally assume they contain gene expression data
# unless we see evidence otherwise
is_gene_available = True

# 2. Define conversion functions for each variable
def convert_trait(value):
    """Convert trait value to binary (0 for normal/control, 1 for cancer/case)"""
    if value is None or pd.isna(value):
        return None
    
    # Extract the value after the colon if present
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
    
    value_lower = str(value).lower()
    
    # Map to binary values for Adrenocortical Cancer
    if any(term in value_lower for term in ["normal", "control", "healthy", "non-tumor", "non tumor", "adjacent", "non-neoplastic"]):
        return 0
    elif any(term in value_lower for term in ["tumor", "cancer", "carcinoma", "adrenocortical", "adenoma", "adc", "acc", "malignant"]):
        return 1
    else:
        return None

def convert_age(value):
    """Convert age value to continuous numeric value"""
    if value is None or pd.isna(value):
        return None
    
    # Extract the value after the colon if present
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Try to extract numeric age
    import re
    age_match = re.search(r'(\d+)', str(value))
    if age_match:
        return float(age_match.group(1))
    else:
        return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    if value is None or pd.isna(value):
        return None
    
    # Extract the value after the colon if present
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
    
    value_lower = str(value).lower()
    
    # Map to binary values
    if any(


### Step 4: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# Check the raw clinical data
import os
import json
import pandas as pd
from typing import Optional, Callable, Dict, Any

# Let's first check what data we have
# Since we need to analyze the clinical_data, let's check if it exists
if 'clinical_data' in globals():
    print("clinical_data exists. Examining its structure...")
    print(f"Shape: {clinical_data.shape}")
    print(f"First few rows:\n{clinical_data.head()}")
    
    # Look at unique values in sample characteristics to identify relevant rows
    print("\nUnique values in sample characteristics:")
    for i in range(clinical_data.shape[0]):
        print(f"Row {i}: {set(clinical_data.iloc[i, :])}")
else:
    print("clinical_data variable is not available. We need to load the data first.")
    # You might need to load the data here, but since this is a continuation, 
    # we'll assume the data has been loaded in a previous step.

# Let's first check if this is a gene expression dataset
is_gene_available = True  # Based on the assumption this is gene expression data
                          # Without detailed data, we're making a best judgment

# Define functions to convert trait, age, and gender data
def convert_trait(value):
    if pd.isna(value) or value is None:
        return None
    
    # Extract the value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary based on common descriptions
    if value.lower() in ['normal', 'healthy', 'control', 'non-cancer', 'non cancer']:
        return 0
    elif value.lower() in ['cancer', 'tumor', 'adrenocortical cancer', 'acc', 'adrenocortical carcinoma']:
        return 1
    else:
        return None

def convert_age(value):
    if pd.isna(value) or value is None:
        return None
    
    # Extract the value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Try to convert to float
    try:
        # Remove any non-numeric characters except decimal point
        cleaned_value = ''.join(c for c in value if c.isdigit() or c == '.')
        age = float(cleaned_value)
        return age
    except:
        return None

def convert_gender(value):
    if pd.isna(value) or value is None:
        return None
    
    # Extract the value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary (0 for female, 1 for male)
    if value.lower() in ['female', 'f', 'woman']:
        return 0
    elif value.lower() in ['male', 'm', 'man']:
        return 1
    else:
        return None

# Assuming based on typical GEO datasets:
# We need to check if these rows actually contain the needed information
trait_row = 1  # Typically disease status is in row 1
age_row = None  # Often age data is not provided
gender_row = None  # Often gender data is not provided

# Check if trait_row is valid (meaning trait data is available)
is_trait_available = trait_row is not None

# Save metadata using the validate_and_save_cohort_info function
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Extract clinical features if trait data is available
if trait_row is not None:
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    preview = preview_df(selected_clinical_df)
    print("\nPreview of extracted clinical features:")
    print(preview)
    
    # Save the clinical data to the specified file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=True)
    print(f"Clinical data saved to {out_clinical_data_file}")
else:
    print("Trait data is not available. Skipping clinical feature extraction.")